In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
print("Pytorch version:", torch.__version__)

Pytorch version: 2.9.1+cpu


In [4]:
#2 TRANSFORM
transform = transforms.Compose([
    transforms.Resize((28, 28)),  # resize to 28x28
    transforms.Grayscale(),       # conver to 1 channel
    transforms.ToTensor()
])

In [9]:
#3 Load Dataset (our own images cat/dog)
train_dataset = datasets.ImageFolder("myImages/train", transform=transform)
test_dataset = datasets.ImageFolder("myImages/test", transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("Training samples:", len(train_dataset))
print("Testing samples:", len(test_dataset))
print("Class names:", train_dataset.classes)

Training samples: 10
Testing samples: 2
Class names: ['cats', 'dogs']


In [15]:
#4 Simple Neural NETWORK
class SimpleClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 10) 

    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x
model = SimpleClassifier()
print(model)

SimpleClassifier(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)


In [16]:
#5 LOSS & Optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [17]:
#6 Training Loop
EPOCHS = 5
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        outputs = model(images)
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{EPOCHS} -> Loss: {total_loss:.4f}")

Epoch 1/5 -> Loss: 2.3972
Epoch 2/5 -> Loss: 1.7574
Epoch 3/5 -> Loss: 1.3826
Epoch 4/5 -> Loss: 1.1008
Epoch 5/5 -> Loss: 0.9063


In [19]:
#7 Evaluation
correct = 0
total = 0
model.eval()

with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        predictions = torch.argmax(outputs, dim=1)
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

print("Test Accuracy:", correct / total)

Test Accuracy: 0.5


In [28]:
#8 Predict a single image
example_img, example_label = test_dataset[1]
model.eval()
with torch.no_grad():
    output = model(example_img.unsqueeze(0))
    pred = torch.argmax(output)

print("True Label:", example_label)
print("Predicted:", pred.item())

True Label: 1
Predicted: 0
